In [1]:
!pip install pytorch_lightning efficientnet_pytorch colorednoise torchlibrosa audiomentations

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import sys
import random
import os
import math
import time

import numpy as np
import pandas as pd
import librosa
import librosa.display
from collections import defaultdict
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import label_ranking_average_precision_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function, Variable
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.core import LightningModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from audiomentations import Compose, AddGaussianNoise, Gain, TimeStretch, PitchShift

In [3]:
def set_random_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
BASE_INPUT_DIR = './input/'
TRAIN_INPUT_DIR = os.path.join(BASE_INPUT_DIR, 'rfcx-species-audio-detection/train')
TEST_INPUT_DIR = os.path.join(BASE_INPUT_DIR, 'rfcx-species-audio-detection/test')
OUTPUT_DIR = './output'
MODELS_DIR = './models'

train_tp = pd.read_csv(os.path.join(BASE_INPUT_DIR, 'rfcx-species-audio-detection/train_tp.csv'))
submission = pd.read_csv(os.path.join(BASE_INPUT_DIR, 'rfcx-species-audio-detection/sample_submission.csv'))

In [5]:
sys.path.append(os.path.join(BASE_INPUT_DIR, 'panns-sed'))

from models import Cnn14_DecisionLevelAtt, AttBlock, init_layer
from pytorch_utils import move_data_to_device, do_self_mixup

In [6]:
# LRAP. Instance-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = (scores.sum(-1) / labels.sum(-1)).mean()
    return score.item()


# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(labels, preds):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))]), device=DEVICE).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()


def _to_one_hot(y, n_dims, dtype=torch.cuda.FloatTensor):
    scatter_dim = len(y.size())
    y_tensor = y.type(torch.cuda.LongTensor).view(*y.size(), -1)
    zeros = torch.zeros(*y.size(), n_dims).type(dtype)
        
    return zeros.scatter(scatter_dim, y_tensor, 1)


class LSEP2(Function): 
    """
    Autograd function of LSEP loss. Appropirate for multi-label
    - Reference: Li+2017
      https://arxiv.org/pdf/1704.03135.pdf
      Code-ref: https://github.com/Mipanox/Bird_cocktail/blob/196e9404a4f7022d1e56433112f581b34a334e53/utils.py#L332
    """
    
    @staticmethod
    def forward(ctx, input, target):
        batch_size = target.size()[0]
        label_size = target.size()[1]

        ##
        positive_indices = target.gt(0).float()
        negative_indices = target.eq(0).float()
        
        ## summing over all negatives and positives
        loss = 0.
        for i in range(input.size()[0]):
            pos = positive_indices[i].nonzero()
            neg = negative_indices[i].nonzero()
            pos_examples = input[i, pos]
            neg_examples = torch.transpose(input[i, neg], 0, 1)
            loss += torch.sum(torch.exp(neg_examples - pos_examples))
        
        loss = torch.log(1 + loss)
        
        ctx.save_for_backward(input, target)
        ctx.loss = loss
        ctx.positive_indices = positive_indices
        ctx.negative_indices = negative_indices
        
        return loss

    # This function has only a single output, so it gets only one gradient 
    @staticmethod
    def backward(ctx, grad_output):
        dtype = torch.cuda.FloatTensor
        input, target = ctx.saved_tensors
        N = input.size()[1]
        loss = Variable(ctx.loss, requires_grad=False)
        positive_indices = ctx.positive_indices
        negative_indices = ctx.negative_indices

        fac  = -1 / loss
        grad_input = torch.zeros(input.size()).type(dtype)
        
        scale = grad_input.size(0), -1
        phot = _to_one_hot(positive_indices.nonzero()[:, 1].view(*scale), N)
        nhot = _to_one_hot(negative_indices.nonzero()[:, 1].view(*scale), N)

        scale = (len(phot), *nhot.size())
        diffs = torch.sum(phot - nhot.expand(scale), dim=2)
        grads_input = (Variable(diffs * torch.exp(-input * diffs)) * (grad_output * fac))
        
        return grad_input, None, None


class LSEPLoss2(nn.Module): 
    def __init__(self): 
        super(LSEPLoss2, self).__init__()
        
    def forward(self, output_dict, target): 
        framewise_output = output_dict["framewise_output"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)
        return LSEP2.apply(output_dict['clipwise_output'], target) + 0.5*LSEP2.apply(clipwise_output_with_max, target)


class ImprovedPANNsLoss(nn.Module):

    def __init__(self):
        super().__init__()
        self.normal_loss = nn.BCELoss()
        self.bce = nn.BCELoss()

    def forward(self, output_dict, target):
        framewise_output = output_dict["framewise_output"]
        clipwise_output_with_max, _ = framewise_output.max(dim=1)

        normal_loss = self.bce(output_dict['clipwise_output'], target)
        auxiliary_loss = self.bce(clipwise_output_with_max, target)
        return normal_loss + 0.5 * auxiliary_loss

# Constants & Config

In [7]:
DEVICE= "cuda:0"

NUM_SPECIES = 24

FMIN = 40.0
FMAX = 24000.0

SR = 48000
FRAME_SIZE = 1024
HOP_LENGTH = 320
N_MELS = 64

CLIP_DURATION = 60

In [8]:
cfg = {
    'preprocess': {
        'frame_size': FRAME_SIZE,
        'hop_length': HOP_LENGTH,
        'sub_segment_duration': 6,

        'do_mixup': True,
        'mixup_alpha': 0.2
    },
    'training': {
        'n_folds': 5,
        'batch_size': 32,
        'epochs': 100,
        'max_lr': 1e-3,
        'early_stopping': 15,
    },
    'model': {
        'base_model': 'cnn14_att',
    },
    'inference': {
        'segment_stride': 6
    }
}

# Preprocessing

In [9]:
class RFCXDataset(torch.utils.data.Dataset):

    def __init__(self, samples, configs, is_train):
        self.samples = samples
        self.cfg = configs
        self.is_train = is_train
        self.augment = Compose([
            AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.03, p=0.7),
            Gain(min_gain_in_db=-15, max_gain_in_db=15, p=0.7),
        ])
        
    def _load_audio(self, recording_id):
        filepath = os.path.join(TRAIN_INPUT_DIR, recording_id + '.flac')
        data, _ = librosa.load(filepath, sr=SR)
        return data
    
    def _cut_wav(self, audio_data, sample):
        tmin = sample['t_min']
        tmax = sample['t_max']
        sub_segment_duration = cfg['preprocess']['sub_segment_duration']

        if self.is_train:
            if tmax - tmin < sub_segment_duration:
                min_left = max(0.0, tmax - sub_segment_duration)
                max_left = min(tmin, CLIP_DURATION - sub_segment_duration)
            else:
                shrinkage = (tmax - tmin) - sub_segment_duration
                min_left = tmin
                max_left = tmin + shrinkage
            left_cut = np.random.uniform(low=min_left, high=max_left)
        else:
            if tmax - tmin < sub_segment_duration:
                extension = max(0.0, sub_segment_duration - (tmax - tmin))/2
                left_extend = extension
                if tmax + extension > CLIP_DURATION:
                    left_extend += tmax + extension - CLIP_DURATION
                left_cut = max(0.0, tmin - left_extend)
            else:
                shrinkage = (tmax - tmin) - sub_segment_duration
                left_cut = tmin + shrinkage/2

        left_cut_sample = int(np.floor(left_cut * SR))
        right_cut_sample = left_cut_sample + sub_segment_duration*SR
        cut = audio_data[left_cut_sample:right_cut_sample]
        assert len(cut) == sub_segment_duration*SR
        return cut
    
    def _one_hot(self, idx):
        target = np.zeros(NUM_SPECIES, dtype=np.float32)
        sparse_label = self.samples.species_id.iloc[idx]
        target[sparse_label] = 1
        return target

    def __len__(self):
        return self.samples.shape[0]

    def __getitem__(self, idx: int):
        sample = self.samples.iloc[idx, :]
        data = self._load_audio(sample.recording_id)
        wav, target = self._cut_wav(data, sample), self._one_hot(idx)
        if self.is_train:
            wav = self.augment(samples=wav, sample_rate=SR)
        return wav, target

In [10]:
class Mixup(object):

    def __init__(self, mixup_alpha, random_seed=1234):
        """Mixup coefficient generator.
        """
        self.mixup_alpha = mixup_alpha
        self.random_state = np.random.RandomState(random_seed)

    def get_lambda(self, batch_size):
        """Get mixup random coefficients.
        Args:
          batch_size: int
        Returns:
          mixup_lambdas: (batch_size,)
        """
        mixup_lambdas = [
            self.random_state.beta(self.mixup_alpha, self.mixup_alpha, 1)[0]
            for _ in range(0, batch_size)
        ]
        return np.array(mixup_lambdas)

In [11]:
# melspec = librosa.feature.melspectrogram(ds[4][0], sr=SR)
# melspec = librosa.power_to_db(melspec)
# librosa.display.specshow(melspec, x_axis='time', y_axis='mel')

# Model

In [12]:
def create_model(load_pretrained=True):
    with_mixup = cfg['preprocess']['do_mixup']
    args = {
        'sample_rate': SR,
        'window_size': FRAME_SIZE,
        'hop_size': HOP_LENGTH,
        'mel_bins': N_MELS,
        'fmin': FMIN, 
        'fmax': FMAX,
        'classes_num': 527,
    }
    base_model = Cnn14_DecisionLevelAtt(**args)
    if load_pretrained:
        checkpoint = torch.load(os.path.join(BASE_INPUT_DIR, 'panns-sed/Cnn14_DecisionLevelAtt_mAP=0.425.pth'))
        base_model.load_state_dict(checkpoint['model'])

    base_model.att_block = AttBlock(2048, NUM_SPECIES, activation="sigmoid")
    base_model.att_block.init_weights()
    init_layer(base_model.fc1)
    return base_model

In [13]:
class LightModel(LightningModule):

    def __init__(self, model, train_samples, val_samples):
        super().__init__()
        self.model = model
        self.train_samples = train_samples
        self.val_samples = val_samples
        self.loss_fn = ImprovedPANNsLoss()
        self.mixup_augmenter = Mixup(cfg['preprocess']['mixup_alpha'])

    def forward(self, batch, mixup_lambda=None, mixup_indices=None):
        return self.model(batch, mixup_lambda=mixup_lambda, mixup_indices=mixup_indices)
    
    def train_dataloader(self):
        return DataLoader(
            RFCXDataset(self.train_samples, cfg, True),
            batch_size=cfg['training']['batch_size'],
            shuffle=True,
            pin_memory=True,
            drop_last=False,
        )

    def val_dataloader(self):
        return DataLoader(
            RFCXDataset(self.val_samples, cfg, False), 
            batch_size=cfg['training']['batch_size'],
            shuffle=False,
            pin_memory=True,
            drop_last=False
        )

    def training_step(self, batch, batch_idx):
        y, target = [x.to(DEVICE) for x in batch]
        mixup_lambda = None
        mixup_indices = None
        if cfg['preprocess']['do_mixup']:
            num_samples = len(y)
            mixup_lambda = self.mixup_augmenter.get_lambda(batch_size=num_samples)
            mixup_lambda = move_data_to_device(mixup_lambda, DEVICE)
            mixup_indices = torch.randperm(num_samples, device=DEVICE)
            target = do_self_mixup(target, mixup_lambda, mixup_indices)
        output = model(y, mixup_lambda=mixup_lambda, mixup_indices=mixup_indices)
        bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        self.log('train_loss', loss, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        y, target = [x.to(self.device) for x in batch]
        output = model(y)
        bceLoss = self.loss_fn(output, target)
        loss = bceLoss
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        lwap = LWLRAP(target, output['clipwise_output'])
        self.log('LwAP', lwap, on_epoch=True, prog_bar=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(model.parameters(), lr=cfg['training']['max_lr'])
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-8)
#         scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, T_mult=2, eta_min=1e-8)
        lr_scheduler = {'scheduler': scheduler}
#         reduce_on_plateau = {
#             'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=7, min_lr=1e-8, verbose=True),
#             'monitor': 'LwAP',
#             'interval': 'epoch',
#             'frequency': 1,
#         }
        return [optimizer], [lr_scheduler]

In [14]:
!rm -r ./output/lightning_logs/*

In [15]:
# models_path = os.path.join(MODELS_DIR, 'panns_weights/output')
# model_files = os.listdir(models_path)
# file_by_fold = []
# for i in range(cfg['training']['n_folds']):
#     for f in model_files:
#         if f.startswith(f'weights-{i}'):
#             file_by_fold.append(f)
#             break

species_and_songtypes = train_tp['species_id'].astype(str) + '_' + train_tp['songtype_id'].astype(str)

In [ ]:
skf = StratifiedKFold(n_splits=cfg['training']['n_folds'])
random_seeds = [6895, 9177, 6725, 9874, 1703]
print('Configs:', cfg)

for fold, (train_idx, val_idx) in enumerate(skf.split(train_tp, species_and_songtypes)):
    set_random_seeds(random_seeds[fold])

    train_samples = train_tp.loc[train_idx, :]
    val_samples = train_tp.loc[val_idx, :]

#     base_model = create_model(load_pretrained=False)
#     model = LightModel(base_model, train_samples, val_samples).to(DEVICE)
#     weight_path = os.path.join(models_path, file_by_fold[fold])
#     state_dict = torch.load(weight_path, map_location=DEVICE)["state_dict"]
#     model.load_state_dict(state_dict)
    base_model = create_model(load_pretrained=True)
    model = LightModel(base_model, train_samples, val_samples).to(DEVICE)
    model.train()

    checkpoint_callback = ModelCheckpoint(
        filepath=os.path.join(OUTPUT_DIR, '{epoch:02d}-{LwAP:.4f}'),
        save_top_k=1,
        verbose=1,
        monitor='LwAP',
        mode='max',
        prefix=f'weights-{fold}',
        save_weights_only=True
    )
    early_stop_callback = EarlyStopping(
       monitor='LwAP',
       min_delta=0.00,
       patience=cfg['training']['early_stopping'],
       verbose=False,
       mode='max'
    )
    logger = TensorBoardLogger(save_dir=OUTPUT_DIR, name='lightning_logs')

    print(f'Traing fold [{fold}]')
    trainer = Trainer(
        max_epochs=cfg['training']['epochs'],
        gradient_clip_val=1,
        logger=logger,
        checkpoint_callback=checkpoint_callback,
        callbacks=[early_stop_callback],
        limit_val_batches=0.2,
        gpus=int(torch.cuda.is_available())
    )

    trainer.fit(model)
    del model
    del base_model

Configs: {'preprocess': {'frame_size': 1024, 'hop_length': 320, 'sub_segment_duration': 6, 'do_mixup': True, 'mixup_alpha': 0.2}, 'training': {'n_folds': 5, 'batch_size': 32, 'epochs': 100, 'max_lr': 0.001, 'early_stopping': 15}, 'model': {'base_model': 'cnn14_att'}, 'inference': {'segment_stride': 6}}


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /home/jupyter/output exists and is not empty. With save_top_k=1, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Traing fold [0]



  | Name    | Type                   | Params
---------------------------------------------------
0 | model   | Cnn14_DecisionLevelAtt | 80.9 M
1 | loss_fn | ImprovedPANNsLoss      | 0     
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

Epoch 28: LwAP was not in top 1


Epoch 29: LwAP was not in top 1


Epoch 30: LwAP was not in top 1


Epoch 31: LwAP was not in top 1


Epoch 32: LwAP reached 0.90885 (best 0.90885), saving model to /home/jupyter/output/weights-1-epoch=32-LwAP=0.9089.ckpt as top 1


In [ ]:
!zip -r pann_logs.zip ./output/lightning_logs
!zip panns_weights.zip ./output/weights-*.ckpt
!gsutil cp ./panns_weights.zip gs://tn-kaggle-data/